In [1]:
import os

CLR_SUBDIR = "cavalcante/rust_xml_image/"
CLM_SUBDIR = "cavalcante/miner_img_xml/"

def get_all_with_ext(subdir, ext):
    return [f for f in os.listdir(subdir) if f.endswith(ext)]

clr_imgs = get_all_with_ext(CLR_SUBDIR, ".jpg")
clm_imgs = get_all_with_ext(CLM_SUBDIR, ".jpg")

print(len(clr_imgs))
print(len(clm_imgs))

285
257


In [4]:
from PIL import Image

def check_img_list_size(img_list, subdir):
    freq = {} 
    for img in img_list:
        im = Image.open(os.path.join(subdir, img))
        width, height = im.size
        dim = str(width) + "x" + str(height)
        if (dim in freq): 
            freq[dim] += 1
        else: 
            freq[dim] = 1
    return freq
        
check_img_list_size(clr_imgs, CLR_SUBDIR)

{'4000x2250': 275, '2549x3506': 7, '2250x4000': 3}

In [5]:
check_img_list_size(clm_imgs, CLM_SUBDIR)

{'4000x2250': 257}

In [6]:
from xml.etree import ElementTree

clr_xmls = get_all_with_ext("cavalcante/rust_xml_image", ".xml")
clm_xmls = get_all_with_ext("cavalcante/miner_img_xml", ".xml")

print(len(clr_xmls))
print(len(clm_xmls))

285
257


In [5]:
# Refered to https://machinelearningmastery.com/how-to-train-an-object-detection-model-with-keras/

# function to extract bounding boxes from an annotation file
def extract_boxes(filename):
    # load and parse the file
    tree = ElementTree.parse(filename)
    # get the root of the document
    root = tree.getroot()
    # extract each bounding box
    boxes = list()
    for box in root.findall('.//bndbox'):
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)
        coors = [xmin, ymin, xmax, ymax]
        boxes.append(coors)
    # extract image dimensions
    width = int(root.find('.//size/width').text)
    height = int(root.find('.//size/height').text)
    return boxes, width, height


In [7]:
# extract details form annotation file
boxes, w, h = extract_boxes('cavalcante/miner_img_xml/bicho_mineiro0.xml')
# summarize extracted details
print(boxes, w, h)

[[2339, 163, 3299, 987], [2261, 1192, 2791, 1625], [739, 1034, 1252, 1373]] 4000 2250
